In [ ]:
import os
import sys
import PIL
import pickle
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append('stylegan2encoder')

import dnnlib
import pretrained_networks
import dnnlib.tflib as tflib
from encoder.generator_model import Generator

In [ ]:
# Make sure the images are rotated properly:
# https://medium.com/@ageitgey/the-dumb-reason-your-fancy-computer-vision-app-isnt-working-exif-orientation-73166c7d39da
def exif_transpose(img):
    if not img:
        return img
    
    exif_orientation_tag = 274
    
    if not hasattr(img, '_getexif'):
        return img
    if not isinstance(img._getexif(), dict):
        return img
    if not exif_orientation_tag in img._getexif():
        return img
    
    exif_data = img._getexif()
    orientation = exif_data[exif_orientation_tag]
    
    # Handle orientation:
    if orientation == 1:
        pass
    elif orientation == 2:
        img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    elif orientation == 3:
        img = img.rotate(180)
    elif orientation == 4:
        img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
    elif orientation == 5:
        img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
    elif orientation == 6:
        img = img.rotate(-90, expand=True)
    elif orientation == 7:
        img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
    elif orientation == 8:
        img = img.rotate(90, expand=True)

    return img

def get_files_with_ext(path, extensions):
    files = os.listdir(path)
    files = [os.path.join(path,f) for f in files]
    files = [f for f in files if os.path.isfile(f)]
    files = [f for f in files if f.endswith(tuple(extensions))]
    return files

def rotate_images(src_path, dest_path):
    image_files = get_files_with_ext(src_path, ['.jpg','.png','.jpeg'])

    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    
    for file in image_files:
        # Load the image
        image = PIL.Image.open(file)
        # Fix the orientation using EXIT data
        image = exif_transpose(image)
        # Save the image
        new_file = os.path.splitext(os.path.basename(file))[0]
        new_file = new_file + '.png'
        new_file = os.path.join(dest_path, new_file)
        image.save(new_file)        

def align_images(src_path, dest_path):
    !python stylegan2encoder/align_images.py $src_path $dest_path
    
def resize_images(src_path, dest_path, max_size=1024):
    image_files = get_files_with_ext(src_path, ['.jpg','.png','.jpeg'])

    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    
    for file in image_files:
        # Load the image
        image = PIL.Image.open(file)
        # ---------------------------
        # Resize the image
        # ---------------------------
        width, height = image.size
        if width > height:
            if width > max_size:
                new_width = int(max_size)
                new_height = int(height * (max_size / width))
                image = image.resize((new_width, new_height), PIL.Image.BILINEAR)
        else:
            if height > max_size:
                new_width = int(width * (max_size / height))
                new_height = int(max_size)
                image = image.resize((new_width, new_height), PIL.Image.BILINEAR)
        # ---------------------------
        
        # Save the image
        new_file = os.path.splitext(os.path.basename(file))[0]
        new_file = new_file + '.png'
        new_file = os.path.join(dest_path, new_file)
        image.save(new_file)
        
def compute_image_latent_vectors(src_path, dest_path):
    !python stylegan2encoder/project_images.py $src_path $dest_path --video=True --video-mode=2
    

def get_latent_vectors(path):
    files = get_files_with_ext(path, '.npy')
    latent_vectors = [np.load(f) for f in files]
    return latent_vectors


In [ ]:
def load_model(model_weights):
    _G, _D, Gs = pretrained_networks.load_networks(model_weights)
    generator = Generator(Gs, batch_size=1, randomize_noise=False)
    model = {
        'generator': generator
    }
    return model

def generate_image(latent_vector, model):
    latent_vector = latent_vector.reshape((1, 18, 512))
    model['generator'].set_dlatents(latent_vector)
    img_array = model['generator'].generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    # img = img.resize((256, 256))
    return img

def show(latent_vector, model):
    plt.subplots()
    plt.imshow(generate_image(latent_vector, model))
    plt.show()

In [ ]:
def mix_latents(latent_vector_a, latent_vector_b, t):
    return latent_vector_a + (latent_vector_b - latent_vector_a) * t

def get_latent_directions(path):
    files = get_files_with_ext(path, '.npy')
    
    latent_directions = {}
    
    for file in files:
        name = os.path.splitext(os.path.basename(file))[0]
        latent_directions[name] = np.load(file)
    
    return latent_directions

def move(latent_vector, feature, dist, latent_directions):
    return latent_vector + latent_directions[feature] * dist

# Top to bottom example:

In [ ]:
# Verify that images are upright
rotate_images('raw_images','rotated_images')

# Crop to faces in images
align_images('rotated_images','aligned_images')

# Make images be no bigger than 1024
resize_images('aligned_images','resized_images',1024)

# Compute latent representation of images
# compute_image_latent_vectors('resized_images','processed_images')

# Load the latent representations
latent_reps = get_latent_vectors('processed_images')

stylegan2weights = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
model = load_model(stylegan2weights)

# Draw the latent
show(latent_reps[0], model)

# Mix the first two latents
show(mix_latents(latent_reps[0],latent_reps[1], 0.5), model)

# Load the latent direction basis vectors
latent_directions = get_latent_directions('stylegan2encoder/latent_directions')
print('-- Available Latent Directions --')
for k in latent_directions.keys():
    print('\t',k)

# Make them happy
show(move(latent_reps[0], 'smile', -30.0, latent_directions), model)